In [149]:
import heapq, itertools


In [150]:
def read_inp(fn):
    with open(fn) as f:
        lines = [ln.strip() for ln in f.read().splitlines() if ln.strip()!='']
    if not lines:
        raise ValueError("input empty")
    try:
        n = int(lines[0])
    except:
        raise ValueError("first non-empty line must be board size (3 or 4)")
    if len(lines) < 1 + n + 1:
        raise ValueError(f"need 1 + {n} + 1 lines (size + board rows + player). Got {len(lines)}")
    b = []
    for i in range(n):
        row = lines[1+i]
        row_chars = [c for c in row if c in "XO."]
        if len(row_chars) != n:
            parts = row.split()
            if len(parts)==n and all(len(p)==1 and p in "XO." for p in parts):
                row_chars = parts
            else:
                raise ValueError(f"row {i+1} must contain exactly {n} symbols X/O/. (found: {row!r})")
        b.append(row_chars)
    p = lines[1+n].upper()
    if p not in ("X","O"):
        raise ValueError("player line must be X or O")
    return n, b, p

In [151]:
def lines_of(b, n):
    L = []
    for r in b:
        L.append(list(r))
    for c in range(n):
        L.append([b[r][c] for r in range(n)])
    L.append([b[i][i] for i in range(n)])
    L.append([b[i][n-1-i] for i in range(n)])
    return L

In [152]:
def win(b, n, q):
    t = [q]*n
    for ln in lines_of(b,n):
        if ln == t:
            return True
    return False

In [153]:

def heuristic(b, n, root):
    opp = "O" if root=="X" else "X"
    L = lines_of(b,n)
    hp = sum(1 for ln in L if opp not in ln)    # lines possible for root
    ho = sum(1 for ln in L if root not in ln)   # lines possible for opponent
    return hp - ho

In [154]:
def to_key(b):
    return tuple("".join(r) for r in b)

def gen_moves(b, n, p):
    for r in range(n):
        for c in range(n):
            if b[r][c]=='.':
                nb = [row[:] for row in b]
                nb[r][c] = p
                yield nb, (r,c)


In [155]:
def best_first(fn_in, fn_steps="steps.txt", fn_out="output.txt", max_expansions=20000):
    open(fn_steps,"w").close()
    open(fn_out,"w").close()
    n,b,root = read_inp(fn_in)
    if win(b,n,"X") or win(b,n,"O"):
        with open(fn_out,"a") as f:
            f.write("Initial board already terminal:\n")
            for r in b: f.write("".join(r)+"\n")
        return
    ctr = itertools.count()
    start_key = to_key(b)
    start_h = heuristic(b,n,root)
    start = {"key":start_key, "b":b, "next":root, "par":None, "mv":None, "d":0, "h":start_h}
    pq = [(-start_h, next(ctr), start)]
    seen = set()
    exp = 0
    nd = None
    while pq and exp < max_expansions:
        pr, _, nd = heapq.heappop(pq)
        exp += 1
        with open(fn_steps,"a") as fs:
            fs.write(f"Step {exp}  h={nd['h']}  next={nd['next']} d={nd['d']}\n")
            for r in nd['b']:
                fs.write("".join(r)+"\n")
            fs.write("-"*10+"\n")
        key = nd['key']
        nxt = nd['next']
        if (key, nxt) in seen:
            continue
        seen.add((key, nxt))
        for nb, mv in gen_moves(nd['b'], n, nxt):
            k = to_key(nb)
            mover = nxt
            if win(nb, n, mover):
                child = {"key":k, "b":nb, "next": ("O" if mover=="X" else "X"), "par":nd, "mv":mv, "d": nd['d']+1, "h": heuristic(nb,n,root)}
                path = []
                cur = child
                while cur is not None:
                    path.append(cur)
                    cur = cur['par']
                path = list(reversed(path))
                with open(fn_out,"a") as fo:
                    fo.write(f"Win found for {mover} at step {exp}. Sequence:\n")
                    for node in path:
                        fo.write(f"depth={node['d']} next={node['next']}\n")
                        for r in node['b']:
                            fo.write(''.join(r)+ '\n')
                        fo.write('-'*6 + '\n')
                return
            child = {"key":k, "b":nb, "next": ("O" if nxt=="X" else "X"), "par":nd, "mv":mv, "d": nd['d']+1, "h": heuristic(nb,n,root)}
            heapq.heappush(pq, (-child['h'], next(ctr), child))
    with open(fn_out,"a") as fo:
        fo.write(f"No winning sequence found within {exp} expansions.\n")
        fo.write("Last examined board:\n")
        if nd:
            for r in nd['b']:
                fo.write(''.join(r)+'\n')


In [157]:
if __name__ == "__main__":
    # default filename is input.txt; you can change it or pass through an environment in Colab.
    best_first("/content/sample_data/input.txt", "/content/sample_data/steps.txt", "/content/sample_data/output.txt")
    print("Done. Check steps.txt and output.txt")



Done. Check steps.txt and output.txt
